# SPARQL Queries - Edinburgh Ladies’ Debating Society - LadiesDebating-KG


This notebook enables to perform several SPARQL queries to the LadiesDebating-KG Knowlege Graph (ladies_debating.ttl). It includes two types RDF queries:

 - Querying the KG locally: Using rdflib.plugins.sparql
 - Querying the KG from an Apache Fuseki Server: Using SPARQLWrapper

The LadiesDebating-KG implements the [NLS-Ontology](https://francesnlp.github.io/NLS-ontology/doc/index-en.html), which has the following [Data Model](https://francesnlp.github.io/NLS-ontology/doc/dataModel.png)

The LadiesDebating-KG (ladies_debating.ttl) can be download from [Zenodo](https://zenodo.org/record/6686072#.YrNDUpDMJHY). 

In general terms this NLS-Ontology can be described as follows:
1. A Digital Collection can have several Series.
2. A Serie can have several Volumes (Books). 
3. A Serie can have several Editors, Publishers and reference books
4. A Volume has several Pages
5. A Page has text 
6. A Serie can have a Supplement 
7. A Supplement can have several Editors, Publishers and reference books
8. A Supplement can have several Volumes


Example of values of the LadiesDebating-KG properties:


    
    mmsid                                                   9938524913804340
    editor name                              Edinburgh Essay Society (Edinburgh)
    editor_date                                                          0.0
    genre                                                         periodical
    language                                                             eng
    metsXML                                               103655648-mets.xml
    termsOfAddress                                                       0.0
    numberOfPages                                                        316
    numberOfWords                                                          1
    permanentURL                            https://digital.nls.uk/103655648
    physicalDescription    10 v. ([2], 286; [2], iv, 292; [2], iv, 286; [...
    place                                                          Edinburgh
    publisher                                                              0
    referencedBy                                                         0.0
    shelfLocator                                                       U.431
    altoXML                                  103655648/alto/104346954.34.xml
    serie SubTitle                                                        0.0
    text                                                             UL.u^\,
    page number                                                                4
    volume title                        attempt. A literary magazine Volume 2
    volumeId                                                       103655648
    year                                                                1865
    serie number                                                           0
    part                                                                   0
    serie title             Edinburgh Ladies’ Debating Society_attempt. A ...
    publisherPersons                                                       0
    numberOfVolumes                                                       10
    volume number                                                          2


### Loading the necessary libraries

In [1]:
import rdflib
from rdflib.extras.external_graph_libs import rdflib_to_networkx_multidigraph
import networkx as nx
import matplotlib.pyplot as pl
from rdflib import Graph, ConjunctiveGraph, Namespace, Literal
from rdflib.plugins.sparql import prepareQuery

In [2]:
import networkx as nx
import matplotlib.pyplot as plt
from SPARQLWrapper import SPARQLWrapper, JSON

In [3]:
import matplotlib.pyplot as plt
plt.rcParams['figure.figsize'] = [15, 10]

### Functions

In [4]:
def plot_resource(results):

    G = rdflib_to_networkx_multidigraph(results)
    pos = nx.spring_layout(G, scale=3)
    edge_labels = nx.get_edge_attributes(G, 'r')
    nx.draw_networkx_edge_labels(G, pos, edge_labels=edge_labels)
    nx.draw(G, with_labels=True)
    plt.show()

### Type 1:  Working with the Knowlege Graph locally

In [5]:
g = Graph()
## Modify the path if necessary 
g.parse("./results/ladies_debating.ttl", format="ttl") 

<Graph identifier=N41bd4be1eeaa47a0acf5710e81371969 (<class 'rdflib.graph.Graph'>)>

Query 1: List all the resources with the property nls:editor in the LadiesDebating-KG 

In [6]:
nls = Namespace("https://w3id.org/nls#")

q1 = prepareQuery('''
  SELECT ?Serie WHERE { 
    ?Serie nls:editor ?FullName. 
  }
  ''',
  initNs = { "nls": nls}
)


for r in g.query(q1):
      print(r.Serie)

https://w3id.org/nls/i/Serie/9938524913804340


Query 2: Same query but asking more information about the resources obtained. 

In [7]:
q2 = prepareQuery('''
  SELECT ?Subject ?Editor WHERE { 
    ?Subject nls:editor ?Editor.
  } 
  ''',
  initNs = { "nls": nls}
)

for r in g.query(q2):
  print(r.Subject, r.Editor)

https://w3id.org/nls/i/Serie/9938524913804340 https://w3id.org/nls/i/Person/EdinburghEssaySocietyEdinburgh


Query 3: Obtaining just the editors names

In [8]:
q3= prepareQuery('''
SELECT DISTINCT ?name
    WHERE {
     ?instance nls:editor ?Editor.
     ?Editor nls:name ?name .
    }
    ''',
  initNs = { "nls": nls}
)

for r in g.query(q3):
      print(r.name)

Edinburgh Essay Society (Edinburgh)


In [9]:
res=g.query(q3)
a=list(res)[0]
a.name

rdflib.term.Literal('Edinburgh Essay Society (Edinburgh)', datatype=rdflib.term.URIRef('http://www.w3.org/2001/XMLSchema#string'))

Query 4: Same query asking for the first 10 resources with the properity nls.name

In [10]:
q4 = prepareQuery('''
  SELECT ?Subject ?Name WHERE { 
    ?Subject nls:name ?Name.
  } 
  LIMIT 10
  ''',
  initNs = { "nls": nls}
)

for r in g.query(q4):
    print(r.Subject, r.Name)

https://w3id.org/nls/i/Person/EdinburghEssaySocietyEdinburgh Edinburgh Essay Society (Edinburgh)


Query 5: Obtain the resources titles which have been printed in Glasgow

In [11]:
from rdflib import XSD
q5 = prepareQuery('''
SELECT ?name ?place
WHERE {
 ?Subject nls:printedAt ?place;
          nls:title ?name.
}

  ''',
  initNs = { "nls": nls}
)

for r in g.query(q5, initBindings = {'?place' : Literal('Edinburgh', datatype=XSD.string)}):
    print(r.name)

Ladies' Edinburgh magazine,1875
attempt. A literary magazine,1865


Query 7: Asking for resources which genre is periodical

In [12]:
from rdflib import XSD
q7 = prepareQuery('''
  SELECT ?Subject WHERE { 
    ?Subject nls:genre ?Genre.
  } 
  ''',
    initNs = { "nls": nls}
)

for r in g.query(q7, initBindings = {'?Genre' : Literal('periodical', datatype=XSD.string)}):
  print(r.Subject)

https://w3id.org/nls/i/Serie/9927010233804340
https://w3id.org/nls/i/Serie/9938524913804340


Query 8: Counting the number of "Serie" resources

In [13]:
q8 = prepareQuery('''
    SELECT ?serie
    WHERE {
       ?serie rdf:type nls:Serie .
    }
    ''',
  initNs = { "nls": nls}
)

print(len(g.query(q8)))


2


q8bis: Obtaining Series URIS and collection name

In [14]:
q8 = prepareQuery('''
    SELECT ?serie ?collection
    WHERE {
       ?serie rdf:type nls:Serie .
       ?serie nls:collection ?collection .
    }
    ''',
  initNs = { "nls": nls}
)

for r in g.query(q8):
    print(r.serie, r.collection)

https://w3id.org/nls/i/Serie/9927010233804340 Edinburgh Ladies’ Debating Society
https://w3id.org/nls/i/Serie/9938524913804340 Edinburgh Ladies’ Debating Society


q8c: Obtaining the number of volumes

In [15]:
q8 = prepareQuery('''
    SELECT ?serie
    WHERE {
       ?volume rdf:type nls:Volume .
    }
    ''',
  initNs = { "nls": nls}
)

print(len(g.query(q8)))

16


q8d: Obtaining the URIS of volumes

In [16]:
q8 = prepareQuery('''
    SELECT ?volume ?title
    WHERE {
       ?volume rdf:type nls:Volume .
       ?volume nls:title ?title .
    }
    ''',
  initNs = { "nls": nls}
)

for r in g.query(q8):
    print(r.volume, r.title)

https://w3id.org/nls/i/Volume/9927010233804340_103655658 Ladies' Edinburgh magazine Volume 1
https://w3id.org/nls/i/Volume/9927010233804340_103655659 Ladies' Edinburgh magazine Volume 2
https://w3id.org/nls/i/Volume/9927010233804340_103655660 Ladies' Edinburgh magazine Volume 3
https://w3id.org/nls/i/Volume/9927010233804340_103655661 Ladies' Edinburgh magazine Volume 4
https://w3id.org/nls/i/Volume/9927010233804340_103655662 Ladies' Edinburgh magazine Volume 5
https://w3id.org/nls/i/Volume/9927010233804340_103655663 Ladies' Edinburgh magazine Volume 6
https://w3id.org/nls/i/Volume/9938524913804340_103655648 attempt. A literary magazine Volume 2
https://w3id.org/nls/i/Volume/9938524913804340_103655649 attempt. A literary magazine Volume 3
https://w3id.org/nls/i/Volume/9938524913804340_103655650 attempt. A literary magazine Volume 4
https://w3id.org/nls/i/Volume/9938524913804340_103655651 attempt. A literary magazine Volume 5
https://w3id.org/nls/i/Volume/9938524913804340_103655652 attem

Query to obtain the number of volumes per serie, and also printing the name of each serie. 

In [17]:
q8 = prepareQuery('''
    SELECT ?serie ?title ?nv
    WHERE {
       ?serie rdf:type nls:Serie .
       ?serie nls:title ?title .
       ?serie nls:numberOfVolumes ?nv .
     }
    ''',
  initNs = { "nls": nls}
)

for r in g.query(q8):
    print("Serie Title: %s, Num volume %s" %(r.title, r.nv))

Serie Title: Ladies' Edinburgh magazine,1875, Num volume 6
Serie Title: attempt. A literary magazine,1865, Num volume 10


Query 9: Priting the first 10 "Pages" uris

In [51]:
q9 = prepareQuery('''
    SELECT ?page
    WHERE {
       ?page rdf:type nls:Page .
    }
    LIMIT 10
    ''',
  initNs = { "nls": nls}
)

for r in g.query(q9):
    print(r.page)

https://w3id.org/nls/i/Page/9927010233804340_103655658_1
https://w3id.org/nls/i/Page/9927010233804340_103655658_10
https://w3id.org/nls/i/Page/9927010233804340_103655658_100
https://w3id.org/nls/i/Page/9927010233804340_103655658_101
https://w3id.org/nls/i/Page/9927010233804340_103655658_102
https://w3id.org/nls/i/Page/9927010233804340_103655658_103
https://w3id.org/nls/i/Page/9927010233804340_103655658_104
https://w3id.org/nls/i/Page/9927010233804340_103655658_105
https://w3id.org/nls/i/Page/9927010233804340_103655658_106
https://w3id.org/nls/i/Page/9927010233804340_103655658_107


Query 10: Otaining the text and the page-number of the first 10 pages

In [52]:
q10 = prepareQuery('''
    SELECT *
    WHERE {
       ?page rdf:type nls:Page .
       ?page nls:text ?text . 
       ?page nls:number ?number .

    }
    LIMIT 5
    ''',
  initNs = { "nls": nls}
)

for r in g.query(q10):
    print("---")
    print(r.text, r.number)

---
 1
---
2 The Ladies Edinhuagh Magazine. may be the light of our lives, but Avhich no act of our own will can bring back. It is not till the distinction has been appreciated between nature as it is and nature as we make it to be, between that which we see and that which ' having not seen, we love,' that any branch of art can be reckoned in its proper value." The writer of these remarks then goes on to contrast the matter-of-fact aspect of the source of our knowledge with the aspect of philosophy, art, and religion. The former takes our knowledge to be exclusively the result of the action of human thought; the latter admits the co-operation of nature by the transmission of images. The view of art, taking its stand on this basis, involves the absolute fusion of thought and things. The habitual interpretation of natural events by the analogy of human design, is but an anticipation of, and a step towards, such a true conquest of nature. This habitual interpretation is a proof that, " to

Query 11: Obtaining the title of the first 10 resources 

In [53]:
q11 = prepareQuery('''
  SELECT ?serie ?title
    WHERE {
    ?serie rdf:type nls:Serie .
    ?serie nls:title ?title .
    }
    LIMIT 10
    ''',
  initNs = { "nls": nls}
)

for r in g.query(q11):
      print(r.title)

Ladies' Edinburgh magazine,1875
attempt. A literary magazine,1865


Query 12: Obtaining each serie number, serie year, serie mmsid, serie language 

In [54]:
q12 = prepareQuery('''
SELECT ?enum ?s ?y ?nv ?mmsid ?title ?subtitle  ?lang WHERE {
       ?s a nls:Serie ;
            nls:number ?enum ;
            nls:mmsid ?mmsid ; 
            nls:publicationYear ?y ; 
            nls:numberOfVolumes ?nv ;
            nls:title ?title ;
            nls:subtitle ?subtitle ;
            nls:language ?lang .
               
    }
''',
  initNs = { "nls": nls}
)

for r in g.query(q12):
    #print(r.enum, r.y, r.nv, r.mmsid, r.lang, r.title, r.subtitle)
    print(r.title, r.subtitle)

Ladies' Edinburgh magazine,1875 0.0
attempt. A literary magazine,1865 0.0


In [56]:
q12 = prepareQuery('''
SELECT ?title WHERE {
       ?s a nls:Volume ;
            nls:title ?title ;
        
               
    }
''',
  initNs = { "nls": nls}
)

for r in g.query(q12):
    #print(r.enum, r.y, r.nv, r.mmsid, r.lang, r.title, r.subtitle)
    print(r.title)

Ladies' Edinburgh magazine Volume 1
Ladies' Edinburgh magazine Volume 2
Ladies' Edinburgh magazine Volume 3
Ladies' Edinburgh magazine Volume 4
Ladies' Edinburgh magazine Volume 5
Ladies' Edinburgh magazine Volume 6
attempt. A literary magazine Volume 2
attempt. A literary magazine Volume 3
attempt. A literary magazine Volume 4
attempt. A literary magazine Volume 5
attempt. A literary magazine Volume 6
attempt. A literary magazine Volume 7
attempt. A literary magazine Volume 8
attempt. A literary magazine Volume 9
attempt. A literary magazine Volume 10
attempt. A literary magazine Volume 1 and Select writings


Query 13: Obtaining the text of pages, in which appear the string "Ladies". 

In [55]:
q13 = prepareQuery('''
SELECT * WHERE {
       ?page a nls:Page .
       ?page nls:text ?text .
      FILTER regex(?text, "^central")
           }

''',
  initNs = { "nls": nls}
)

for r in g.query(q13):
    print(r)
    print("---")

Query 14: Counting the number of Volumes

In [21]:
q14 = prepareQuery('''
SELECT (COUNT (DISTINCT ?v) as ?count)
    WHERE {
        ?serie a nls:Serie .
        ?serie nls:hasPart ?v .
        }

''',
  initNs = { "nls": nls}
)

for r in g.query(q14):
    print("-Number of Volumes--")
    print(r)

-Number of Volumes--
(rdflib.term.Literal('16', datatype=rdflib.term.URIRef('http://www.w3.org/2001/XMLSchema#integer')),)


Query 15: Counting the number of Pages

In [22]:
q15 = prepareQuery('''
SELECT (COUNT (DISTINCT ?p) as ?count)
    WHERE {
        ?volume a nls:Volume .
        ?volume nls:hasPart ?p .
        }

''',
  initNs = { "nls": nls}
)

for r in g.query(q15):
    print("-Number of PAGES--")
    print(r)


-Number of PAGES--
(rdflib.term.Literal('6354', datatype=rdflib.term.URIRef('http://www.w3.org/2001/XMLSchema#integer')),)


Query 16: Counting the number of Series

In [23]:
q16 = prepareQuery('''
SELECT (COUNT (DISTINCT ?s) as ?count)
    WHERE {
        ?serie a nls:Serie .
        ?serie nls:mmsid ?s .
        }

''',
  initNs = { "nls": nls}
)

for r in g.query(q16):
    print("-Number of Series--")
    print(r)


-Number of Series--
(rdflib.term.Literal('2', datatype=rdflib.term.URIRef('http://www.w3.org/2001/XMLSchema#integer')),)


Query 17: Obtaining the uri, year, number of volumes and number of pages of the first 3 series resources

In [24]:
q17 =prepareQuery('''
SELECT ?uri ?year ?vnum ?numPages ?numWords
        WHERE {
        ?uri a nls:Page .
        ?uri nls:numberOfWords ?numWords. 
        ?v nls:hasPart ?uri .
        ?v nls:number ?vnum .
        ?v nls:numberOfPages ?numPages .
        ?e nls:hasPart ?v .
        ?e nls:publicationYear ?year.
        
        }
         LIMIT 3
        ''',
  initNs = { "nls": nls}
)

for r in g.query(q17):
    print("-Number of Series--")
    print(r.uri, r.year, r.vnum, r.numPages)

-Number of Series--
https://w3id.org/nls/i/Page/9927010233804340_103655658_1 1875 1 402
-Number of Series--
https://w3id.org/nls/i/Page/9927010233804340_103655658_10 1875 1 402
-Number of Series--
https://w3id.org/nls/i/Page/9927010233804340_103655658_100 1875 1 402


In [25]:
#G = rdflib_to_networkx_multidigraph(result)

# Plot Networkx instance of RDF Graph
#pos = nx.spring_layout(G, scale=2)
#edge_labels = nx.get_edge_attributes(G, 'r')
#nx.draw_networkx_edge_labels(G, pos, edge_labels=edge_labels)
#nx.draw(G, with_labels=True)

#if not in interactive mode for 
#plt.show()


### Type 2: Connecting with FUSEKI and using SPARQLWrapper

**Previously the knowlege graph (ladies_debating.ttl) needs to be uploaded to Apache Fuseki Server** 

Query 18: Basic query

In [26]:
sparql = SPARQLWrapper("http://localhost:3030/ladies_debating/sparql")
sparql.setQuery("""
    SELECT ?subject ?predicate ?object WHERE {   ?subject ?predicate ?object } LIMIT 5 
""")
sparql.setReturnFormat(JSON)
results = sparql.query().convert()

results

for result in results["results"]["bindings"]:
    print(result["subject"]["value"], result["predicate"]["value"], result["object"]["value"] )


https://w3id.org/nls/i/Page/9938524913804340_103655655_373 http://www.w3.org/1999/02/22-rdf-syntax-ns#type https://w3id.org/nls#Page
https://w3id.org/nls/i/Page/9938524913804340_103655655_373 https://w3id.org/nls#altoXML 103655655/alto/104323660.34.xml
https://w3id.org/nls/i/Page/9938524913804340_103655655_373 https://w3id.org/nls#text The Attempt. 363 It was Sunday morning,—a bright and breezy September day; and the fine old Carillion was ringing, Mary, Joseph, and George, as the bells were called, whose voices were known and loved far out at sea. How often had Raoul heard them in his mind, as now in truth. While avoiding the main thoroughfare, he sped up the narrow lanes, past flowery hedges and scattered cottages towards the massive grey church, which stood alone in a grassy valley, somewhat retired from the little town that lay between it and the sea. And, down the little turfy hill, he could see Gabrielle coming the straight way from the farm. Her freshest bloom seemed glowing in 

Query 19: Obtaining each Serie uri, year and number of Serie. 

In [27]:
sparql = SPARQLWrapper("http://localhost:3030/ladies_debating/sparql")
sparql.setQuery("""
    PREFIX nls: <https://w3id.org/nls#>
    SELECT ?serie ?year ?snum WHERE {
       ?serie a nls:Serie ;
              nls:number ?snum ;
              nls:publicationYear ?year 
    }

""")
sparql.setReturnFormat(JSON)
results = sparql.query().convert()
r=results["results"]["bindings"]
for i in r:
    print(i["serie"]["value"], i["year"]["value"], i["snum"]["value"])

https://w3id.org/nls/i/Serie/9927010233804340 1875 1
https://w3id.org/nls/i/Serie/9938524913804340 1865 0


Query 20: Obtaining publication year, number, title, subtitle, printedAt, physical description, mmsid, shelflocator, number of volumes of the resource with the uri: https://w3id.org/nls/i/Serie/9927010233804340

In [28]:
sparql = SPARQLWrapper("http://localhost:3030/ladies_debating/sparql")
uri="<https://w3id.org/nls/i/Serie/9927010233804340>"
query="""
PREFIX nls: <https://w3id.org/nls#>
SELECT ?publicationYear ?num ?title ?subtitle ?printedAt ?physicalDescription ?mmsid ?shelfLocator ?numberOfVolumes  WHERE {
       %s nls:publicationYear ?publicationYear ;
          nls:number ?num;
          nls:title ?title;
          nls:subtitle ?subtitle ;
          nls:printedAt ?printedAt;
          nls:physicalDescription ?physicalDescription;
          nls:mmsid ?mmsid;
          nls:shelfLocator ?shelfLocator;
          nls:numberOfVolumes ?numberOfVolumes. 
          
               
}
""" % (uri)
sparql.setQuery(query)
sparql.setReturnFormat(JSON)
results = sparql.query().convert()
r=results["results"]["bindings"]
for i in r:
    print(i["publicationYear"]["value"], i["num"]["value"], i["title"]["value"], i["subtitle"]["value"],\
          i["printedAt"]["value"], i["physicalDescription"]["value"], i["mmsid"]["value"], i["shelfLocator"]["value"],\
          i["numberOfVolumes"]["value"])

1875 1 Ladies' Edinburgh magazine,1875 0.0 Edinburgh v. ; 21 cm. 9927010233804340 U.393 6


Query 21: Obtaining the number of volumes of the serie with the uri: https://w3id.org/nls/i/Serie/9938524913804340 

In [29]:
sparql = SPARQLWrapper("http://localhost:3030/ladies_debating/sparql")
uri="<https://w3id.org/nls/i/Serie/9938524913804340>"
query="""
PREFIX nls: <https://w3id.org/nls#>
SELECT (COUNT (DISTINCT ?v) as ?count)
    WHERE {
        %s nls:hasPart ?v.
    	?v ?b ?c
}
""" % (uri)
sparql.setQuery(query)
sparql.setReturnFormat(JSON)
results = sparql.query().convert()
results["results"]["bindings"][0]["count"]["value"]


'10'

In [30]:
sparql = SPARQLWrapper("http://localhost:3030/ladies_debating/sparql")
uri="<https://w3id.org/nls/i/Serie/9927010233804340>"
query="""
PREFIX nls: <https://w3id.org/nls#>
SELECT (COUNT (DISTINCT ?v) as ?count)
    WHERE {
        %s nls:hasPart ?v.
    	?v ?b ?c
}
""" % (uri)
sparql.setQuery(query)
sparql.setReturnFormat(JSON)
results = sparql.query().convert()
results["results"]["bindings"][0]["count"]["value"]


'6'

In [21]:
sparql = SPARQLWrapper("http://localhost:3030/ladies_debating/sparql")
uri="<https://w3id.org/nls/i/Serie/9927010233804340>"
query="""
PREFIX nls: <https://w3id.org/nls#>
SELECT ?v ?title ?vnum
    WHERE {
        %s nls:hasPart ?v.
    	?v nls:title ?title .
        ?v nls:title ?vnum .
}
""" % (uri)
sparql.setQuery(query)
sparql.setReturnFormat(JSON)
results = sparql.query().convert()
r=results["results"]["bindings"]
for i in r:
        print(i["v"]["value"], i["vnum"]["value"])

https://w3id.org/nls/i/Volume/9927010233804340_103655663 Ladies' Edinburgh magazine Volume 6
https://w3id.org/nls/i/Volume/9927010233804340_103655660 Ladies' Edinburgh magazine Volume 3
https://w3id.org/nls/i/Volume/9927010233804340_103655658 Ladies' Edinburgh magazine Volume 1
https://w3id.org/nls/i/Volume/9927010233804340_103655661 Ladies' Edinburgh magazine Volume 4
https://w3id.org/nls/i/Volume/9927010233804340_103655659 Ladies' Edinburgh magazine Volume 2
https://w3id.org/nls/i/Volume/9927010233804340_103655662 Ladies' Edinburgh magazine Volume 5


Query 22: Giving the volume with the uri: <https://w3id.org/nls/i/Volume/9927010233804340_103655658>, this query obtaians its pages uris and numbers. 

In [31]:
sparql = SPARQLWrapper("http://localhost:3030/ladies_debating/sparql")
uri="<https://w3id.org/nls/i/Volume/9927010233804340_103655658>"
query="""
PREFIX nls: <https://w3id.org/nls#>
SELECT ?v ?vnum ?part  WHERE {
       %s nls:hasPart ?v .
       ?v nls:number ?vnum ; 
          OPTIONAL {?v nls:part ?part; }
     
               
} 
""" % (uri)
sparql.setQuery(query)
sparql.setReturnFormat(JSON)
results = sparql.query().convert()
r=results["results"]["bindings"]
for i in r:
    if "part" in i:
        print(i["v"]["value"], i["vnum"]["value"], i["part"]["value"])
    else:
        print(i["v"]["value"], i["vnum"]["value"])

https://w3id.org/nls/i/Page/9927010233804340_103655658_138 138
https://w3id.org/nls/i/Page/9927010233804340_103655658_59 59
https://w3id.org/nls/i/Page/9927010233804340_103655658_183 183
https://w3id.org/nls/i/Page/9927010233804340_103655658_289 289
https://w3id.org/nls/i/Page/9927010233804340_103655658_159 159
https://w3id.org/nls/i/Page/9927010233804340_103655658_206 206
https://w3id.org/nls/i/Page/9927010233804340_103655658_350 350
https://w3id.org/nls/i/Page/9927010233804340_103655658_19 19
https://w3id.org/nls/i/Page/9927010233804340_103655658_321 321
https://w3id.org/nls/i/Page/9927010233804340_103655658_352 352
https://w3id.org/nls/i/Page/9927010233804340_103655658_305 305
https://w3id.org/nls/i/Page/9927010233804340_103655658_176 176
https://w3id.org/nls/i/Page/9927010233804340_103655658_252 252
https://w3id.org/nls/i/Page/9927010233804340_103655658_96 96
https://w3id.org/nls/i/Page/9927010233804340_103655658_272 272
https://w3id.org/nls/i/Page/9927010233804340_103655658_92 92


Query 23: Giving the volume with the uri: <https://w3id.org/nls/i/Volume/9927010233804340_103655658>, this query obtaians: number, title, part, metsXML, volume Id, permanent URL, number of Pages

In [32]:
sparql = SPARQLWrapper("http://localhost:3030/ladies_debating/sparql")
uri="<https://w3id.org/nls/i/Volume/9927010233804340_103655658>"
query="""
PREFIX nls: <https://w3id.org/nls#>
SELECT ?num ?title ?part ?metsXML ?volumeId ?permanentURL ?numberOfPages  WHERE {
       %s nls:number ?num ;
          nls:title ?title;
          nls:metsXML ?metsXML;
          nls:volumeId ?volumeId;
          nls:permanentURL ?permanentURL;
          nls:numberOfPages ?numberOfPages;
       OPTIONAL {%s nls:part ?part. }
      
               
}
""" % (uri, uri)
sparql.setQuery(query)
sparql.setReturnFormat(JSON)
results = sparql.query().convert()
r=results["results"]["bindings"]
for i in r:
    print(i)

{'num': {'type': 'literal', 'datatype': 'http://www.w3.org/2001/XMLSchema#int', 'value': '1'}, 'title': {'type': 'literal', 'value': "Ladies' Edinburgh magazine Volume 1"}, 'metsXML': {'type': 'literal', 'value': '103655658-mets.xml'}, 'volumeId': {'type': 'literal', 'datatype': 'http://www.w3.org/2001/XMLSchema#int', 'value': '103655658'}, 'permanentURL': {'type': 'literal', 'value': 'https://digital.nls.uk/103655658'}, 'numberOfPages': {'type': 'literal', 'value': '402'}}


Query 23: Giving the volume with the uri: https://w3id.org/nls/i/Volume/9927010233804340_103655658, this query obtains the number of pages. 


In [33]:
sparql = SPARQLWrapper("http://localhost:3030/ladies_debating/sparql")
uri="<https://w3id.org/nls/i/Volume/9927010233804340_103655658>"
query="""
PREFIX nls: <https://w3id.org/nls#>
SELECT (COUNT (DISTINCT ?p) as ?count)
    WHERE {
        %s nls:hasPart ?p .
        ?p a nls:Page 
        
} 
""" % (uri)
      
            
sparql.setQuery(query)
sparql.setReturnFormat(JSON)
results = sparql.query().convert()
results

{'head': {'vars': ['count']},
 'results': {'bindings': [{'count': {'type': 'literal',
     'datatype': 'http://www.w3.org/2001/XMLSchema#integer',
     'value': '402'}}]}}

In [34]:
sparql = SPARQLWrapper("http://localhost:3030/ladies_debating/sparql")
uri="<https://w3id.org/nls/i/Volume/9927010233804340_103655658>"
query="""
    PREFIX nls: <https://w3id.org/nls#>
       PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
       SELECT (count (DISTINCT ?a) as ?count)
       WHERE {
            %s nls:hasPart ?b .
            ?b a nls:Page .
            ?b nls:altoXML ?a.
      }


""" % (uri)
            
sparql.setQuery(query)
sparql.setReturnFormat(JSON)
results = sparql.query().convert()
results = sparql.query().convert()
results["results"]["bindings"][0]["count"]["value"]

'402'

Query 27: This query obtains the first 3 pages with the following properties:  uri, year, title, serie number, volume number, volume number, number of Pages, metsXML, number of page, text, number of Words, number of Pages

In [35]:
sparql = SPARQLWrapper("http://localhost:3030/ladies_debating/sparql")
sparql.setQuery("""
PREFIX nls: <https://w3id.org/nls#> 
SELECT ?uri ?year ?title ?snum ?vnum ?v ?part ?metsXML ?page ?text ?numberOfWords ?numberOfPages
        WHERE {
        ?uri a nls:Page .
        ?uri nls:text ?text .
        ?uri nls:number ?page .
        ?uri nls:numberOfWords ?numberOfWords .
        ?v nls:hasPart ?uri.
        ?v nls:number ?vnum.
        ?v nls:numberOfPages ?numberOfPages .
        ?v nls:metsXML ?metsXML.
        ?s nls:hasPart ?v.
        ?s nls:publicationYear ?year.
        ?s nls:number ?snum.
        ?s nls:title ?title.
        OPTIONAL {?v nls:part ?part; }

        }   
        LIMIT 3

""")
sparql.setReturnFormat(JSON)
results = sparql.query().convert()
r=results["results"]["bindings"]
for i in r:
    print(i["uri"]["value"], i["year"]["value"], i["title"]["value"], i["snum"]["value"], \
          i["v"]["value"], i["vnum"]["value"],i["numberOfPages"]["value"],\
          i["metsXML"]["value"], i["page"]["value"],i["text"]["value"], i["numberOfWords"]["value"]
          
          )
    print("---")


https://w3id.org/nls/i/Page/9938524913804340_103655655_373 1865 attempt. A literary magazine,1865 0 https://w3id.org/nls/i/Volume/9938524913804340_103655655 9 392 103655655-mets.xml 373 The Attempt. 363 It was Sunday morning,—a bright and breezy September day; and the fine old Carillion was ringing, Mary, Joseph, and George, as the bells were called, whose voices were known and loved far out at sea. How often had Raoul heard them in his mind, as now in truth. While avoiding the main thoroughfare, he sped up the narrow lanes, past flowery hedges and scattered cottages towards the massive grey church, which stood alone in a grassy valley, somewhat retired from the little town that lay between it and the sea. And, down the little turfy hill, he could see Gabrielle coming the straight way from the farm. Her freshest bloom seemed glowing in her cheeks again ; her dark velvet bodice, poin^ c V Alencon cap, and rich gold ornaments, were most becoming, as she hastened to him with a bright smil